In [13]:
import pandas as pd
import numpy as np
import pickle

import sys
sys.path.insert(0, "../src")
sys.path.insert(0, "../")
from d2v_recommender import *
from config import config

In [14]:
d2v = D2V_Recommender()
data_dict = d2v.load_data_dict(config.data_dict_path)

In [15]:
data_dict["X_train"].shape, data_dict["y_train"].shape, data_dict["X_test"].shape, data_dict["y_test"].shape

((3202, 50), (3202,), (353, 50), (353,))